# 1_geo_plots

Creatining maps

01/15/2024

update 10/03/24
remade plots on 04/04/25
making geoplots


In [59]:
import os
import pandas as pd
import glob
import numpy as np
import pickle
import plotly.express as px
import plotly.offline as py_offline
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from tqdm import tqdm
import seaborn as sns
from collections import defaultdict, Counter
import censusdata
import json
# !pip install censusdata
from urllib.request import urlopen
import json
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import shape  # Import the shape function
import branca
import branca.colormap as cm

In [60]:
save_prefix = 'geoplots/'

In [61]:
%%time
active_trials = json.load(open('interventional_trials_with_descendants2024-07-26.json','r'))


CPU times: user 25.1 s, sys: 3.61 s, total: 28.7 s
Wall time: 28.7 s


In [62]:
len(active_trials['data']), active_trials['total']


(20894, 20894)

In [63]:
biomarker_trials = []
nonbiomarker_trials = []
years_arr = []

for study in active_trials['data']:
    genes = [g for g in study['biomarkers_new']['inclusion']['TREE']['symbols_dz'] if type(g)==str]
    if len(genes) >0:
        biomarker_trials.append(study)
        year = study['start_date'].split('-')[0]
        years_arr.append(year)
    else:
        nonbiomarker_trials.append(study)
  

In [64]:
len(biomarker_trials),len(nonbiomarker_trials)

(5057, 15837)

In [65]:
Counter(years_arr)

Counter({'2021': 495,
         '2018': 478,
         '2020': 477,
         '2019': 467,
         '2017': 462,
         '2022': 428,
         '2023': 425,
         '2016': 371,
         '2015': 360,
         '2014': 297,
         '2024': 285,
         '2013': 196,
         '2012': 123,
         '2011': 69,
         '2010': 46,
         '2009': 31,
         '2008': 14,
         '2007': 11,
         '2006': 9,
         '2025': 7,
         '2005': 2,
         '2001': 1,
         '2004': 1,
         '1993': 1,
         '2003': 1})

In [66]:
# test = biomarker_trials[0]
# test['sites']

# 1. plot functions
(code from travis)

In [67]:
def create_circle_map(study_list, savefile):
    """
    study_list <list>
    savefile: <str> i.e "../figures/appendix_circle_map.html"
    return: coordinates_count <dict> 
    """
    # make a map with circles for size 
    map_usa = folium.Map(location=[37.0902, -95.7129], zoom_start=4,control=True)

    # Count occurrences of coordinates
    coordinates_count = {}
    coord_list = list()
    for trial in study_list:
        for site in trial["sites"]:
            if site["org_coordinates"] is not None:
                coord = (site["org_coordinates"]["lat"], site["org_coordinates"]["lon"])
                coord_list.append(coord)
                coordinates_count[coord] = coordinates_count.get(coord, 0) + 1

    # Plot circles
    for coord, count in coordinates_count.items():
        folium.CircleMarker(
            location=coord,
            radius=np.log(count) ,  # Adjust size to your preference
            color='blue',
            fill=True,
            fill_color='blue'
        ).add_to(map_usa)

    
    # Save or display the map
    map_usa.save(savefile)
    
    return coordinates_count


def create_cluster_map(coordinates_count, savefile, radius=60):
    """
    clustering the initialy circle map more by location 
    
    coordinates_count: <dict>
    savefile <str> ie,    "../figures/appendix_map_cluster.html"
    radius <int> 
    
    return None
    
    """
    map_usa = folium.Map(location=[37.0902, -95.7129], zoom_start=4)
    marker_cluster = MarkerCluster(maxClusterRadius=radius).add_to(map_usa)  # Adjust the radius as needed

    # Add points to the cluster
    for coord, count in coordinates_count.items():
        folium.Marker(
            location=coord,
            popup=f'Count: {count}',
            icon=None
        ).add_to(marker_cluster)

    # Save or display the map
    map_usa.save(savefile)


# Function to create slightly offset points
def create_offset_points(coord, count):
    lat, lon = coord
    offset = 0.001  # Small offset value
    return [(lat + offset, lon, count // 2), (lat - offset, lon, count // 2)]

def create_cluster_offset_map(coordinates_count, savefile):
    """
    circles on map but more grouped and using offset points
    
    study_list <list>
    savefile: <str> i.e. "../figures/appendix_map_cluster_updated.html"
    return: new_coordinates_count <dict> 
    """    
    # Original map setup
    map_usa = folium.Map(location=[37.0902, -95.7129], zoom_start=4)
    marker_cluster = MarkerCluster(maxClusterRadius=40).add_to(map_usa)

    # Updated data processing
    new_coordinates_count = {}
    for coord, count in coordinates_count.items():
        #if count == 1:
            # Split individual points into two offset points
        for new_coord in create_offset_points(coord, count):
            new_coordinates_count[new_coord[:2]] = new_coord[2]
        #else:
        #    new_coordinates_count[coord] = count

    # Add the updated points to the cluster
    for coord, count in new_coordinates_count.items():
        folium.Marker(
            location=coord,
            popup=f'Count: {count}',
            icon=None
        ).add_to(marker_cluster)

    # Save or display the map
    map_usa.save(savefile)
    
    return new_coordinates_count

def create_heatmap(coordinates_count, savefile,radius=20,blur=15, steps=100,max_zoom=1,
                  colors_arr=['red', 'orange', 'yellow', 'green', 'cyan',
                                        'blue','indigo','black']):
    """
    heat overlaying map, the heatmap is just looks like fuzzy paint colors on the country
    
    colors_arr=['red', 'orange', 'yellow', 'green', 
                                        'blue', 'indigo', 'violet','black']
                                        
    study_list <list>
    savefile: <str> i.e. "../figures/appendix_map_cluster_updated.html"
    return: None
    """  
    
    map_usa = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

    # Prepare data for HeatMap
    heat_data = [[coord[0], coord[1], count] for coord, count in coordinates_count.items()]

    # create gradient
    color_map=cm.LinearColormap(colors=colors_arr[::-1]).to_step(steps)
    
    color_map.caption = "Relative Density of Clinical Trials"  # how do I change fontsize and color here?
    map_usa.add_child(color_map)

    gradient_map=defaultdict(dict)
    for i in range(steps):
        gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)


    # Create a HeatMap with customized parameters
    HeatMap(heat_data, gradient = gradient_map, radius=radius, blur=blur, max_zoom=max_zoom).add_to(map_usa)

    # Save or display the map
    map_usa.save(savefile)

# 2.  make per county heatmap and dot plot maps

In [68]:
%%time
# getting counties
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Assuming counties is your GeoJSON data
gdf_counties = gpd.GeoDataFrame.from_features(counties['features'])


CPU times: user 181 ms, sys: 15.6 ms, total: 197 ms
Wall time: 533 ms


In [69]:
%%time
coord_list = list()
coord_list_phase1 = list()
coord_list_phase2 = list()
coord_list_phase3 = list()
coord_list_phase4 = list()

org_to_coord = defaultdict(set)
coord_to_org = defaultdict(set)

for trial in active_trials['data']:

    for site in trial["sites"]:
        
        
        if site["org_coordinates"] is not None:
            coord = (site["org_coordinates"]["lat"], site["org_coordinates"]["lon"])
            coord_list.append(coord)
            org_to_coord[site['org_name']].add(coord)
            coord_to_org[coord].add(site['org_name'])

            if trial['phase']=='I':
                coord_list_phase1.append(coord)
            elif trial['phase']=='II' or trial['phase']=='I_II':
                coord_list_phase2.append(coord)
            elif trial['phase']=='III'or trial['phase'] =='II_III':
                coord_list_phase3.append(coord)
            elif trial['phase']=='IV':
                coord_list_phase4.append(coord)

print(len(coord_list))

262910
CPU times: user 227 ms, sys: 233 ms, total: 460 ms
Wall time: 459 ms


In [70]:
%%time
# initializing data

# Convert your coordinate tuples to Shapely Points and then to a GeoDataFrame
gdf_points = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in coord_list])

# Set the same coordinate reference system as the counties GeoDataFrame
gdf_points.crs = gdf_counties.crs

# Initialize a dictionary to hold the count of points in each county
county_point_count = {feature['properties']['NAME']: 0 for feature in counties['features']}

print(len(gdf_points.geometry), len(counties['features']))


262910 3221
CPU times: user 1.19 s, sys: 13.6 ms, total: 1.2 s
Wall time: 1.22 s


In [71]:
point_to_count = gdf_points.geometry.value_counts().to_dict()
len(point_to_count)

2230

In [75]:
%%time

#takes a bit

# Check each point to see which county it falls into and increment the count
for point,count in  point_to_count.items():
    # if (idx%1000==0):
    #     print(idx, 'of', len(gdf_points.geometry))
    for feature in counties['features']:
        polygon = shape(feature['geometry'])
        if point.within(polygon):
            county_name = feature['properties']['NAME']
            county_point_count[county_name] += count
            break
            

CPU times: user 2min 11s, sys: 1.39 s, total: 2min 12s
Wall time: 2min 12s


In [76]:
# Write pickle data
with open('geoplots/all_group_interv_county_counts_intervention.pkl', 'wb') as file:
    pickle.dump(county_point_count, file)

In [77]:
%%time
#plotting
# Convert the county_point_count dictionary to a DataFrame
# Assume 'county_fips' is a list of FIPS codes for each county
county_fips = [feature['properties']['GEO_ID'] for feature in counties['features']]
county_names = [feature['properties']['NAME'] for feature in counties['features']]
county_area = [feature['properties']['CENSUSAREA'] for feature in counties['features']]
df_count = pd.DataFrame({
    'fips': county_fips,
    'county_name': county_names,
    'county_area': county_area,
    'point_count': [county_point_count[name] for name in county_names]
})

# Adjust point_count for logarithmic scale (adding 1 to avoid log(0)) scaled by county area
df_count['log_point_count'] = df_count['point_count'].apply(lambda x: np.log10(x + 1))

df_count['log_point_count_scale'] = df_count.apply(lambda row: min(.01,row['log_point_count']/row['county_area']),axis=1)
                                                        

# Plot the choropleth map using the logarithmically scaled data
fig = px.choropleth(df_count, geojson=counties, locations='fips', color='log_point_count_scale',
                    color_continuous_scale="Viridis",
                    range_color=(0, max(df_count['log_point_count_scale'])),
                    scope="usa",
                    featureidkey="properties.GEO_ID",
                    labels={'log_point_count_scale':'Logarithmic Point Count (scaled by county size) '}
                   )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    coloraxis_colorbar_title_font_size=16,  # Adjust the title font size
    coloraxis_colorbar_tickfont_size=14     # Adjust the tick labels font size
)
fig.write_html("geoplots/all_group_interv_choropleth_map_log_scale_by_size.html")

CPU times: user 469 ms, sys: 21.6 ms, total: 491 ms
Wall time: 506 ms


In [78]:
coordinates_count = create_circle_map(active_trials['data'], os.path.join(save_prefix,'all_group_map_circle.html'))
create_heatmap(coordinates_count,  os.path.join(save_prefix,'all_group_map_heatmap.html'),radius=5,blur=1,max_zoom=1)


In [79]:
coordinates_count = create_circle_map(biomarker_trials, os.path.join(save_prefix,'biomarkers_group_map_circle.html'))
create_heatmap(coordinates_count,  os.path.join(save_prefix,'biomarkers_group_map_heatmap.html'),radius=5,blur=1,max_zoom=1)


In [80]:
coordinates_count = create_circle_map(nonbiomarker_trials, os.path.join(save_prefix,'nonbiomarkers_group_map_circle.html'))
create_heatmap(coordinates_count,  os.path.join(save_prefix,'nonbiomarkers_group_map_heatmap.html'),radius=5,blur=1,max_zoom=1)


# Part 3 - figure out if coordinate is urban or not


In [84]:
import shapefile
import geopandas as gpd
shp = shapefile.Reader('geoplots/tl_rd22_us_uac20/tl_rd22_us_uac20.shp') #open the shapefile
all_shapes = shp.shapes() # get all the polygons
all_records = shp.records()
polygons_gpd = gpd.GeoDataFrame(geometry=all_shapes) #convert the polygons into a geodataframe

def is_urban(coord):
    #test chicago IL: (41.878,-87.6298)
    # result = False
    lat,long = coord
    points_gpd = gpd.GeoDataFrame(geometry=gpd.points_from_xy([long], [lat]))  # point coordinates to geopandas dataframe
    pt2poly = gpd.sjoin(points_gpd,polygons_gpd, predicate='within').index_right  # for each point index in the points, it stores the polygon index containing that point
    if len(pt2poly) >= 1:  # if any indicies are found, it's urban
        return True
    else:
        return False


In [85]:
%%time
urban_coords = []
counter_phaseall_urban = 0

for coord,count_trials  in Counter(coord_list).items():
    if is_urban(coord):
        urban_coords.append(coord)
        counter_phaseall_urban+=count_trials


CPU times: user 14.5 s, sys: 153 ms, total: 14.6 s
Wall time: 14.6 s


In [87]:
len(Counter(coord_list))

2230

In [88]:
len(active_trials['data'])

20894

In [89]:
# , with XXX occurring only within one of the top 10 sites. 
coord_to_numtrials = Counter(coord_list)
coord_to_numtrials_sorted_keys = sorted(coord_to_numtrials, key=coord_to_numtrials.get, reverse=True)

#only want top 10 sites
idx = 0
top10_coordlist = []
for coord in coord_to_numtrials_sorted_keys:
    if idx > 9:
        break
    print(coord, coord_to_numtrials[coord],is_urban(coord))
    top10_coordlist.append(coord)
    idx+=1
    
# find out how many trials occur at these areas:
counter_trial = 0
for trial in active_trials['data']:
    trial_intop10 = []
    for site in trial["sites"]:
        if site["org_coordinates"] is not None:
            coord = (site["org_coordinates"]["lat"], site["org_coordinates"]["lon"])
            if coord in top10_coordlist:
                trial_intop10.append(True)
            else:
                trial_intop10.append(False)
    
    if all(trial_intop10):
        counter_trial+=1
                
            
print(counter_trial, len(active_trials['data']), 
      counter_trial/len(active_trials['data']),
      'in top 10 sites')

    

(29.7059, -95.4026) 5448 True
(42.3469, -71.1025) 3473 True
(40.7656, -73.9624) 2976 True
(38.6267, -90.267) 2395 True
(41.5063, -81.6065) 1894 True
(42.341, -71.0948) 1881 True
(33.792, -84.3239) 1840 True
(34.1357, -117.9655) 1836 True
(42.3621, -71.068) 1823 True
(47.6307, -122.3461) 1823 True
4276 20894 0.2046520532210204 in top 10 sites


20.5% of trials occur only  in the 10 ten sites

by phase

In [90]:
%%time
counter_phase1_urban = 0
counter_phase2_urban = 0
counter_phase3_urban = 0
counter_phase4_urban = 0

counter_phaseall_urban = 0

counter_phase1_all = 0
counter_phase2_all = 0
counter_phase3_all = 0
counter_phase4_all = 0


for trial in active_trials['data']:
    trialsites_urban_boollist = []
    for site in trial["sites"]:
        if site["org_coordinates"] is not None:
            coord = (site["org_coordinates"]["lat"], site["org_coordinates"]["lon"])
            # coord_list.append(coord)
            
            if coord in urban_coords:
                trialsites_urban_boollist.append(True)
            else:
                trialsites_urban_boollist.append(False)
    
    if trial['phase']=='I':
        counter_phase1_all+=1
    elif trial['phase']=='II' or trial['phase']=='I_II':
        counter_phase2_all+=1
    elif trial['phase']=='III'or trial['phase'] =='II_III':
        counter_phase3_all+=1
    elif trial['phase']=='IV':
        counter_phase4_all+=1
            
    if all(trialsites_urban_boollist): # all trial locations are in urban areas:
        counter_phaseall_urban+=1
        #if at lease one of them is rurual 
        if trial['phase']=='I':
            counter_phase1_urban+=1
        elif trial['phase']=='II' or trial['phase']=='I_II':
            counter_phase2_urban+=1
        elif trial['phase']=='III'or trial['phase'] =='II_III':
            counter_phase3_urban+=1
        elif trial['phase']=='IV':
            counter_phase4_urban+=1



CPU times: user 1.98 s, sys: 24.1 ms, total: 2.01 s
Wall time: 2.01 s


In [91]:
# print % urban
print('overall')
print(counter_phaseall_urban, len(active_trials['data']), counter_phaseall_urban/len(active_trials['data']))
print('phase I')
print(counter_phase1_urban, counter_phase1_all, counter_phase1_urban/counter_phase1_all)
print('phase II')
print(counter_phase2_urban, counter_phase2_all, counter_phase2_urban/counter_phase2_all)
print('phase III')
print(counter_phase3_urban, counter_phase3_all, counter_phase3_urban/counter_phase3_all)
print('phase 1V')
print(counter_phase4_urban, counter_phase4_all, counter_phase4_urban/counter_phase4_all)



overall
18934 20894 0.906193165502058
phase I
4888 5156 0.948021722265322
phase II
8150 9097 0.8958997471693965
phase III
1516 2036 0.7445972495088409
phase 1V
189 199 0.949748743718593


In [92]:
len(urban_coords)

1661

In [93]:
len(urban_coords)/len(Counter(coord_list))

0.7448430493273542

There are only 2230 unique clinical trial sites (for the over 20,000 clinical trials). For these, only 1661 (or 74.4% of them) are located in urban areas, as defined by the 2020 US Census. As such 94.8% of phase I, 89.6% of phase II, and 74.4% of phase III clinical trials have sites that are solely in urban areas, which tend of cluster in areas. Given clinical trial access is often concentrated in urban areas, a nationwide, systematic approach is needed to ensure equitable access to clinical trials for all patients, regardless of their location.